In [18]:
import mongoengine
import meetup.api
import os
import configparser
import requests
import json
from opencage.geocoder import OpenCageGeocode

Configparser implementation

In [19]:
config = configparser.ConfigParser()
config.read('config/meetup.cfg')
os.environ['MEETUP_API_KEY'] = config['MEETUP']['API_KEY']
os.environ['OPENCAGE_KEY']=config['OPENCAGE']['KEY']

geocode = OpenCageGeocode(os.environ['OPENCAGE_KEY'])

In [3]:
class Event:
    
    def __init__(self,
                params,
                url_path):
        self.params = params
        self.url_path = url_path
        
    
    def get_event(self, *args, **kwargs):
        
        r = requests.get(self.url_path, params=self.params)
        request_string = r.text
        
        return request_string

In [4]:
default_args = dict(
    country='United Stated',
    key= os.environ['MEETUP_API_KEY'],
    topic='Python'
)

url_meetup_request = "https://api.meetup.com/2/open_events"
events = Event(params=default_args, url_path=url_meetup_request)
response = events.get_event()

In [9]:
json_response = json.loads(response)
data = json_response['results']
len(data)

191

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [7]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.5") \
    .getOrCreate()

In [21]:
data[0]['venue']

{'zip': '94102',
 'country': 'us',
 'localized_country_name': 'USA',
 'city': 'San Francisco',
 'address_1': "333 O'Farrell St",
 'name': 'Hilton San Francisco Union Square',
 'lon': -122.41012,
 'id': 26058606,
 'state': 'CA',
 'lat': 37.785732,
 'repinned': True}

In [ ]:
country, city, name, description, event_url, yes_rsvp_count, status = ([] for i in range(7))

for label in data:
    results_geocode = geocode.reverse_geocode(label['group']['group_lat'], label['group']['group_lon'])
    